## 0. Libarary 불러오기 및 경로설정

In [1]:
import os
import pandas as pd
from PIL import Image
import numpy as np
import pytorchtools

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torch.optim import Adam, lr_scheduler
from torch.utils.data.dataset import random_split
from torch.utils.data.sampler import SubsetRandomSampler

from tqdm import tqdm
from torchvision import transforms
from torchvision.transforms import Resize, ToTensor, Normalize

from sklearn.metrics import f1_score

In [2]:
# 테스트 데이터셋 폴더 경로를 지정해주세요.
test_dir = '/opt/ml/input/data/eval'

---

In [3]:
class TrainDataset(Dataset):
    def __init__(self, train_dir, is_Train=True, transform=None):
        super().__init__()
        csv_path = os.path.join(train_dir, "labelled_data.csv")
        self.df = pd.read_csv(csv_path)
        self.image_dir = os.path.join(train_dir, 'images')
        self.transform = transform
        self.image_path = '/opt/ml/input/data/train/images/'+self.df['file_name']

    def __len__(self):
        return len(self.image_path)
        
    def __getitem__(self, idx):
        image_name = self.image_path[idx]

        image = Image.open(image_name)
        
        target = self.df.loc[idx, 'class']
        if self.transform:
            image = self.transform(image)
        
        return image, target
        

In [4]:
class Iv3(nn.Module):
    def __init__(self):
        super().__init__()
        self.model = models.inception_v3(pretrained=True, aux_logits=False)
        
    def forward(self, x):
        x = self.model(x)
        return x

In [5]:
batch_size = 42
epochs = 10
lr = 1e-4

In [6]:
from torchvision import models


model = Iv3()

transform_train = transforms.Compose([
    transforms.Resize(299),
    transforms.CenterCrop(299),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

transform_val = transforms.Compose([
    transforms.Resize(299),
    transforms.CenterCrop(299),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

dataset = TrainDataset('/opt/ml/input/data/train/', is_Train=True, transform=transform_train)
train_loader = DataLoader(dataset, batch_size=batch_size, pin_memory=True, num_workers=4)


# https://hoya012.github.io/blog/DenseNet-Tutorial-2/
validation_ratio = 0.1
random_seed = 10

num_train = len(dataset)
indices = list(range(num_train))
split = int(np.floor(validation_ratio * num_train))

np.random.seed(random_seed)
np.random.shuffle(indices)

train_idx, valid_idx = indices[split:], indices[:split]
train_sampler = SubsetRandomSampler(train_idx)
valid_sampler = SubsetRandomSampler(valid_idx)

train_loader = torch.utils.data.DataLoader(
    dataset, batch_size=batch_size, sampler=train_sampler, num_workers=4
)

valid_loader = torch.utils.data.DataLoader(
    dataset, batch_size=batch_size, sampler=valid_sampler, num_workers=4
)

In [7]:
from torch.optim import AdamW

In [8]:
criterion = nn.CrossEntropyLoss()
optimizer = AdamW(model.parameters(), lr=lr)
scheduler = lr_scheduler.StepLR(optimizer, step_size=2, gamma=0.2)

In [9]:
# https://quokkas.tistory.com/entry/pytorch%EC%97%90%EC%84%9C-EarlyStop-%EC%9D%B4%EC%9A%A9%ED%95%98%EA%B8%B0

class EarlyStopping:
    """주어진 patience 이후로 validation loss가 개선되지 않으면 학습을 조기 중지"""
    def __init__(self, patience=7, verbose=False, delta=0, path='checkpoint.pt'):
        """
        Args:
            patience (int): validation loss가 개선된 후 기다리는 기간
                            Default: 7
            verbose (bool): True일 경우 각 validation loss의 개선 사항 메세지 출력
                            Default: False
            delta (float): 개선되었다고 인정되는 monitered quantity의 최소 변화
                            Default: 0
            path (str): checkpoint저장 경로
                            Default: 'checkpoint.pt'
        """
        self.patience = patience
        self.verbose = verbose
        self.counter = 0
        self.best_score = None
        self.early_stop = False
        self.val_loss_min = np.Inf
        self.delta = delta
        self.path = path

    def __call__(self, val_loss, model):

        score = -val_loss

        if self.best_score is None:
            self.best_score = score
            self.save_checkpoint(val_loss, model)
        elif score < self.best_score + self.delta:
            self.counter += 1
            print(f'EarlyStopping counter: {self.counter} out of {self.patience}')
            if self.counter >= self.patience:
                self.early_stop = True
        else:
            self.best_score = score
            self.save_checkpoint(val_loss, model)
            self.counter = 0

    def save_checkpoint(self, val_loss, model):
        '''validation loss가 감소하면 모델을 저장한다.'''
        if self.verbose:
            print(f'Validation loss decreased ({self.val_loss_min:.6f} --> {val_loss:.6f}).  Saving model ...')
        torch.save(model.state_dict(), self.path)
        self.val_loss_min = val_loss

In [10]:
VIS = 150
model.cuda()

# 모델이 학습되는 동안 trainning loss를 track
train_losses = []
# 모델이 학습되는 동안 validation loss를 track
valid_losses = []
# epoch당 average training loss를 track
avg_train_losses = []
# epoch당 average validation loss를 track
avg_valid_losses = []

# early_stopping object의 초기화
early_stopping = EarlyStopping(patience = 1, verbose = True)

for epoch in range(epochs):
    running_loss = 0.0
    model.train()
    
    for i, (img, label) in enumerate(tqdm(train_loader)):
        optimizer.zero_grad()
        img, label = img.type(torch.FloatTensor).cuda(), label.cuda()
        pred_logit = model(img)
        loss = criterion(pred_logit, label)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
        train_losses.append(loss.item())
        if i % VIS == VIS-1: 
            print('[%d, %5d] loss: %.3f' % (epoch + 1, i + 1, running_loss / VIS))
            running_loss = 0.0
    
    # https://hoya012.github.io/blog/DenseNet-Tutorial-2/
    correct = 0
    total = 0
    for iter, (img, label) in enumerate(valid_loader):
        img, label = img.type(torch.FloatTensor).cuda(), label.cuda()
        pred_logit = model(img)

        _, predicted = torch.max(pred_logit.data, 1)
        total += label.size(0)
        correct += (predicted == label).sum().item()
        loss = criterion(pred_logit, label)
        valid_losses.append(loss.item())
        
    train_loss = np.average(train_losses)
    valid_loss = np.average(valid_losses)
    avg_train_losses.append(train_loss)
    avg_valid_losses.append(valid_loss)
    
    epoch_len = len(str(epochs))


    print_msg = (f'[{epoch:>{epoch_len}}/{epochs:>{epoch_len}}] ' +
                 f'train_loss: {train_loss:.5f} ' +
                 f'valid_loss: {valid_loss:.5f}')

    print(print_msg)

    # clear lists to track next epoch
    train_losses = []
    valid_losses = []

    # early_stopping는 validation loss가 감소하였는지 확인이 필요하며,
    # 만약 감소하였을경우 현제 모델을 checkpoint로 만든다.
    early_stopping(valid_loss, model)

    if early_stopping.early_stop:
        print("Early stopping")
        break


#     print('[%d epoch] Accuracy of the network on the validation images: %d %%' % 
#           (epoch + 1, 100 * correct / total)
#          )
    f1_score_value = f1_score(label.detach().cpu().numpy(), predicted.detach().cpu().numpy(), average='micro')
    print(f'f1 score is : {f1_score_value}')
    print("-"*50)
    
    if epoch < epochs:
        scheduler.step()

# best model이 저장되어있는 last checkpoint를 로드한다.
model.load_state_dict(torch.load('checkpoint.pt'))

 42%|████▏     | 150/360 [00:36<00:48,  4.31it/s]

[1,   150] loss: 1.216


 83%|████████▎ | 300/360 [01:10<00:13,  4.30it/s]

[1,   300] loss: 0.296


100%|██████████| 360/360 [01:24<00:00,  4.24it/s]


...

 83%|████████▎ | 300/360 [01:10<00:13,  4.31it/s]

[4,   300] loss: 0.013


100%|██████████| 360/360 [01:24<00:00,  4.27it/s]


[ 3/10] train_loss: 0.01413 valid_loss: 0.08146
EarlyStopping counter: 1 out of 1
Early stopping


<All keys matched successfully>

In [11]:
model.load_state_dict(torch.load('checkpoint.pt'))

<All keys matched successfully>

## 2. Test Dataset 정의

In [12]:
class TestDataset(Dataset):
    def __init__(self, img_paths, transform):
        self.img_paths = img_paths
        self.transform = transform

    def __getitem__(self, index):
        image = Image.open(self.img_paths[index])

        if self.transform:
            image = self.transform(image)
        return image

    def __len__(self):
        return len(self.img_paths)

## 3. Inference

In [13]:
# meta 데이터와 이미지 경로를 불러옵니다.
submission = pd.read_csv(os.path.join(test_dir, 'info.csv'))
image_dir = os.path.join(test_dir, 'images')

# Test Dataset 클래스 객체를 생성하고 DataLoader를 만듭니다.
image_paths = [os.path.join(image_dir, img_id) for img_id in submission.ImageID]
transform = transforms.Compose([
    Resize((512, 384), Image.BILINEAR),
    ToTensor(),
    Normalize(mean=(0.5, 0.5, 0.5), std=(0.2, 0.2, 0.2)),
])
dataset = TestDataset(image_paths, transform)

loader = DataLoader(
    dataset,
    shuffle=False
)

# 모델을 정의합니다. (학습한 모델이 있다면 torch.load로 모델을 불러주세요!)
device = torch.device('cuda')
# model = MyModel(num_classes=18).to(device)
model.eval()

# 모델이 테스트 데이터셋을 예측하고 결과를 저장합니다.
all_predictions = []
for images in loader:
    with torch.no_grad():
        images = images.to(device)
        pred = model(images)
        pred = pred.argmax(dim=-1)
        all_predictions.extend(pred.cpu().numpy())
submission['ans'] = all_predictions

# 제출할 파일을 저장합니다.
submission.to_csv(os.path.join(test_dir, 'submission_inception_v3.csv'), index=False)
print('test inference is done!')

test inference is done!
